In [8]:
import numpy as np
import tensorflow as tf

np.random.seed(42)
tf.random.set_seed(42)

# Logistic regression without artificial bias

## Without unbiasing

In [16]:
import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
from sklearn.model_selection import  train_test_split
from sklearn.linear_model import LogisticRegression
from aif360.metrics import ClassificationMetric
from aif360.datasets import StandardDataset, dataset
from copy import copy
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
_night = ['22:', '23:', '00:', '01:', '02:', '03:', '04:', '05:', '06:', '07:']

# data preprocessing
data = pd.read_csv('data/police_project.csv')
data = data.drop(['stop_date','county_name', 'driver_age_raw', 'violation_raw', 'stop_outcome', 'drugs_related_stop', 'search_type'], axis=1)
data = data.dropna()
data['stop_time'] = data['stop_time'].apply(lambda x: 0 if any(n in x for n in _night) else 1)
data['driver_gender'] = data['driver_gender'].astype('category').cat.codes
data['driver_race'] = data['driver_race'].astype('category').cat.codes
data['driver_race'] = data['driver_race'].apply(lambda x: 0 if x==3 else 1)
data['violation'] = data['violation'].astype('category').cat.codes
data['search_conducted'] = data['search_conducted'].astype('category').cat.codes
data['is_arrested'] = data['is_arrested'].astype('category').cat.codes # 0 = False, 1 = True
data['stop_duration'] = data['stop_duration'].astype('category').cat.codes

# datasets
X = data[data.columns.difference(['is_arrested'])]
y = data['is_arrested']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = LogisticRegression(max_iter=1000, ).fit(X_train, y_train)
y_pred = clf.predict(X_test)

# ai360
priv_groups = [{'driver_race': 1}]
unpriv_groups = [{'driver_race': 0}]
dataset = pd.concat([X_test, y_test], axis=1)
dataset = StandardDataset(dataset, label_name='is_arrested', favorable_classes=[0], protected_attribute_names=['driver_race'], privileged_classes=[[1]])
classified_dataset = dataset.copy()
classified_dataset.labels = y_pred

# metrics
class_metric = ClassificationMetric(dataset, classified_dataset, unprivileged_groups=unpriv_groups, privileged_groups=priv_groups)
print(  
    ' FOR:', class_metric.false_omission_rate(privileged=1),'\n',
    'FORD:', class_metric.false_omission_rate_difference(),'\n',
    'Accuracy score:', clf.score(X_test, y_test)
    )

 FOR: 0.5253164556962026 
 FORD: -0.5253164556962026 
 Accuracy score: 0.9660918539162747


invalid value encountered in double_scalars
invalid value encountered in double_scalars


## With unbiasing

In [17]:
from aif360.sklearn.preprocessing import Reweighing
coln = ['driver_race'] # stupid workaround for make it working
X_train.index.names = coln
rew = Reweighing(prot_attr='driver_race').fit_transform(X_train, y_train)
un = np.unique(rew[1])
cweights = {1: un[0], 0: un[1]}

In [18]:
# datasets
X = data[data.columns.difference(['is_arrested'])]
y = data['is_arrested']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = LogisticRegression(class_weight=cweights, max_iter=1000).fit(X_train, y_train)
y_pred = clf.predict(X_test)

# ai360
priv_groups = [{'driver_race': 1}]
unpriv_groups = [{'driver_race': 0}]
dataset = pd.concat([X_test, y_test], axis=1)
dataset = StandardDataset(dataset, label_name='is_arrested', favorable_classes=[0], protected_attribute_names=['driver_race'], privileged_classes=[[1]])
classified_dataset = dataset.copy()
classified_dataset.labels = y_pred

# metrics
class_metric = ClassificationMetric(dataset, classified_dataset, unprivileged_groups=unpriv_groups, privileged_groups=priv_groups)
print(  
    ' FOR:', class_metric.false_omission_rate(privileged=1),'\n',
    'FORD:', class_metric.false_omission_rate_difference(),'\n',
    'Accuracy score:', clf.score(X_test, y_test)
    )

 FOR: 0.0 
 FORD: 0.0 
 Accuracy score: 0.9665563490681066


invalid value encountered in double_scalars
invalid value encountered in double_scalars


# Logistic regression with artificial bias

## Without unbiasing

In [19]:
import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
from sklearn.model_selection import  train_test_split
from sklearn.linear_model import LogisticRegression
from aif360.metrics import ClassificationMetric
from aif360.datasets import StandardDataset, dataset
from copy import copy
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
_night = ['22:', '23:', '00:', '01:', '02:', '03:', '04:', '05:', '06:', '07:']

# data preprocessing
data = pd.read_csv('data/police_project.csv')
data = data.drop(['stop_date','county_name', 'driver_age_raw', 'violation_raw', 'stop_outcome', 'drugs_related_stop', 'search_type'], axis=1)
data = data.dropna()
data['stop_time'] = data['stop_time'].apply(lambda x: 0 if any(n in x for n in _night) else 1)
data['driver_gender'] = data['driver_gender'].astype('category').cat.codes
data['driver_race'] = data['driver_race'].astype('category').cat.codes
data['driver_race'] = data['driver_race'].apply(lambda x: 0 if x==3 else 1)
data['violation'] = data['violation'].astype('category').cat.codes
data['search_conducted'] = data['search_conducted'].astype('category').cat.codes
data['is_arrested'] = data['is_arrested'].astype('category').cat.codes # 0 = False, 1 = True
data['stop_duration'] = data['stop_duration'].astype('category').cat.codes

# add bias
def perform_bias(x):
    r_ch_nw = np.random.choice([0,1], p=[.6,.4])
    r_ch_w = 1 - r_ch_nw
    if  (x == 0 and r_ch_nw == 0) or (x == 1 and r_ch_w == 0):
        return True
    return False
        
data['is_arrested'] = data['driver_race'].apply(lambda x: perform_bias(x))

# datasets
X = data[data.columns.difference(['is_arrested'])]
y = data['is_arrested']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = LogisticRegression(max_iter=1000).fit(X_train, y_train)
y_pred = clf.predict(X_test)

# ai360
priv_groups = [{'driver_race': 1}]
unpriv_groups = [{'driver_race': 0}]
dataset = pd.concat([X_test, y_test], axis=1)
dataset = StandardDataset(dataset, label_name='is_arrested', favorable_classes=[0], protected_attribute_names=['driver_race'], privileged_classes=[[1]])
classified_dataset = dataset.copy()
classified_dataset.labels = y_pred

# metrics
class_metric = ClassificationMetric(dataset, classified_dataset, unprivileged_groups=unpriv_groups, privileged_groups=priv_groups)
print(  
    ' FOR:', class_metric.false_omission_rate(privileged=1),'\n',
    'FORD:', class_metric.false_omission_rate_difference(),'\n',
    'Accuracy score:', clf.score(X_test, y_test)
    )

 FOR: 0.6014906253639222 
 FORD: -0.6014906253639222 
 Accuracy score: 0.6016373454102073


## With unbiasing

In [20]:
from aif360.sklearn.preprocessing import Reweighing
coln = ['driver_race'] # stupid workaround for make it working
X_train.index.names = coln
rew = Reweighing(prot_attr='driver_race').fit_transform(X_train, y_train)
un = np.unique(rew[1])
cweights = {1: un[0], 0: un[1]}

In [21]:
# datasets
X = data[data.columns.difference(['is_arrested'])]
y = data['is_arrested']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = LogisticRegression(class_weight=cweights, max_iter=1000).fit(X_train, y_train)
y_pred = clf.predict(X_test)

# ai360
priv_groups = [{'driver_race': 1}]
unpriv_groups = [{'driver_race': 0}]
dataset = pd.concat([X_test, y_test], axis=1)
dataset = StandardDataset(dataset, label_name='is_arrested', favorable_classes=[0], protected_attribute_names=['driver_race'], privileged_classes=[[1]])
classified_dataset = dataset.copy()
classified_dataset.labels = y_pred

# metrics
class_metric = ClassificationMetric(dataset, classified_dataset, unprivileged_groups=unpriv_groups, privileged_groups=priv_groups)
print(  
    ' FOR:', class_metric.false_omission_rate(privileged=1),'\n',
    'FORD:', class_metric.false_omission_rate_difference(),'\n',
    'Accuracy score:', clf.score(X_test, y_test)
    )

 FOR: 0.6014906253639222 
 FORD: -0.2545518498537181 
 Accuracy score: 0.6007664170005226


# Neural Network without artificial bias

## Without unbiasing

In [39]:
import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
from sklearn.model_selection import  train_test_split
from sklearn.linear_model import LogisticRegression
from aif360.metrics import ClassificationMetric
from aif360.datasets import StandardDataset, dataset
from copy import copy
import matplotlib.pyplot as plt
from aif360.sklearn.inprocessing import AdversarialDebiasing
import tensorflow as tf
pd.set_option('display.max_columns', None)
_night = ['22:', '23:', '00:', '01:', '02:', '03:', '04:', '05:', '06:', '07:']

# data preprocessing
data = pd.read_csv('data/police_project.csv')
data = data.drop(['stop_date','county_name', 'driver_age_raw', 'violation_raw', 'stop_outcome', 'drugs_related_stop', 'search_type'], axis=1)
data = data.dropna()
data['stop_time'] = data['stop_time'].apply(lambda x: 0 if any(n in x for n in _night) else 1)
data['driver_gender'] = data['driver_gender'].astype('category').cat.codes
data['driver_race'] = data['driver_race'].astype('category').cat.codes
data['driver_race'] = data['driver_race'].apply(lambda x: 0 if x==3 else 1)
data['violation'] = data['violation'].astype('category').cat.codes
data['search_conducted'] = data['search_conducted'].astype('category').cat.codes
data['is_arrested'] = data['is_arrested'].astype('category').cat.codes # 0 = False, 1 = True
data['stop_duration'] = data['stop_duration'].astype('category').cat.codes

# datasets
X = data[data.columns.difference(['is_arrested'])]
y = data['is_arrested']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

coln = ['driver_race']
X_train.index.names = coln
tf.compat.v1.disable_eager_execution()
advd = AdversarialDebiasing(
    prot_attr='driver_race',
    num_epochs=5,
    batch_size=128,
    classifier_num_hidden_units=100,
    debias=False
    ).fit(X_train,y_train)
y_pred = advd.predict(X_test)

# ai360
priv_groups = [{'driver_race': 1}]
unpriv_groups = [{'driver_race': 0}]
dataset = pd.concat([X_test, y_test], axis=1)
dataset = StandardDataset(dataset, label_name='is_arrested', favorable_classes=[0], protected_attribute_names=['driver_race'], privileged_classes=[[1]])
classified_dataset = dataset.copy()
classified_dataset.labels = y_pred

# metrics
class_metric = ClassificationMetric(dataset, classified_dataset, unprivileged_groups=unpriv_groups, privileged_groups=priv_groups)
print(  
    ' FOR:', class_metric.false_omission_rate(privileged=1),'\n',
    'FORD:', class_metric.false_omission_rate_difference(),'\n',
    'Accuracy score:', advd.score(X_test, y_test)
    )

 FOR: 0.5444444444444444 
 FORD: -0.5444444444444444 
 Accuracy score: 0.9660918539162747


invalid value encountered in double_scalars
invalid value encountered in double_scalars


## With unbiasing

In [40]:
from aif360.sklearn.inprocessing import AdversarialDebiasing
import tensorflow as tf

coln = ['driver_race']
X_train.index.names = coln
tf.compat.v1.disable_eager_execution()
advd = AdversarialDebiasing(
    prot_attr='driver_race',
    num_epochs=5,
    batch_size=128,
    classifier_num_hidden_units=100,
    ).fit(X_train,y_train)
y_pred = advd.predict(X_test)

# ai360
priv_groups = [{'driver_race': 1}]
unpriv_groups = [{'driver_race': 0}]
dataset = pd.concat([X_test, y_test], axis=1)
dataset = StandardDataset(dataset, label_name='is_arrested', favorable_classes=[0], protected_attribute_names=['driver_race'], privileged_classes=[[1]])
classified_dataset = dataset.copy()
classified_dataset.labels = y_pred

# metrics
class_metric = ClassificationMetric(dataset, classified_dataset, unprivileged_groups=unpriv_groups, privileged_groups=priv_groups)
print(  
    ' FOR:', class_metric.false_omission_rate(privileged=1),'\n',
    'FORD:', class_metric.false_omission_rate_difference(),'\n',
    'Accuracy score:', advd.score(X_test, y_test)
    )

 FOR: 0.0 
 FORD: 0.0 
 Accuracy score: 0.9665563490681066


invalid value encountered in double_scalars
invalid value encountered in double_scalars


# Neural network with artificial bias

## Without unbiasing

In [41]:
import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
from sklearn.model_selection import  train_test_split
from sklearn.linear_model import LogisticRegression
from aif360.metrics import ClassificationMetric
from aif360.datasets import StandardDataset, dataset
from copy import copy
import matplotlib.pyplot as plt
from aif360.sklearn.inprocessing import AdversarialDebiasing
import tensorflow as tf
pd.set_option('display.max_columns', None)
_night = ['22:', '23:', '00:', '01:', '02:', '03:', '04:', '05:', '06:', '07:']

# data preprocessing
data = pd.read_csv('data/police_project.csv')
data = data.drop(['stop_date','county_name', 'driver_age_raw', 'violation_raw', 'stop_outcome', 'drugs_related_stop', 'search_type'], axis=1)
data = data.dropna()
data['stop_time'] = data['stop_time'].apply(lambda x: 0 if any(n in x for n in _night) else 1)
data['driver_gender'] = data['driver_gender'].astype('category').cat.codes
data['driver_race'] = data['driver_race'].astype('category').cat.codes
data['driver_race'] = data['driver_race'].apply(lambda x: 0 if x==3 else 1)
data['violation'] = data['violation'].astype('category').cat.codes
data['search_conducted'] = data['search_conducted'].astype('category').cat.codes
data['is_arrested'] = data['is_arrested'].astype('category').cat.codes # 0 = False, 1 = True
data['stop_duration'] = data['stop_duration'].astype('category').cat.codes

# add bias
def perform_bias(x):
    r_ch_nw = np.random.choice([0,1], p=[.6,.4])
    r_ch_w = 1 - r_ch_nw
    if  (x == 0 and r_ch_nw == 0) or (x == 1 and r_ch_w == 0):
        return True
    return False
        
data['is_arrested'] = data['driver_race'].apply(lambda x: perform_bias(x))

# datasets
X = data[data.columns.difference(['is_arrested'])]
y = data['is_arrested']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

coln = ['driver_race']
X_train.index.names = coln
tf.compat.v1.disable_eager_execution()
advd = AdversarialDebiasing(
    prot_attr='driver_race',
    num_epochs=5,
    batch_size=128,
    classifier_num_hidden_units=100,
    debias=False
    ).fit(X_train,y_train)
y_pred = advd.predict(X_test)

# ai360
priv_groups = [{'driver_race': 1}]
unpriv_groups = [{'driver_race': 0}]
dataset = pd.concat([X_test, y_test], axis=1)
dataset = StandardDataset(dataset, label_name='is_arrested', favorable_classes=[0], protected_attribute_names=['driver_race'], privileged_classes=[[1]])
classified_dataset = dataset.copy()
classified_dataset.labels = y_pred

# metrics
class_metric = ClassificationMetric(dataset, classified_dataset, unprivileged_groups=unpriv_groups, privileged_groups=priv_groups)
print(  
    ' FOR:', class_metric.false_omission_rate(privileged=1),'\n',
    'FORD:', class_metric.false_omission_rate_difference(),'\n',
    'Accuracy score:', advd.score(X_test, y_test)
    )

 FOR: 0.5946197740770933 
 FORD: -0.2680891618321954 
 Accuracy score: 0.5938570516170237


## With unbiasing

In [42]:
from aif360.sklearn.inprocessing import AdversarialDebiasing
import tensorflow as tf

coln = ['driver_race']
X_train.index.names = coln
tf.compat.v1.disable_eager_execution()
advd = AdversarialDebiasing(
    prot_attr='driver_race',
    num_epochs=5,
    batch_size=128,
    classifier_num_hidden_units=100,
    ).fit(X_train,y_train)
y_pred = advd.predict(X_test)

# ai360
priv_groups = [{'driver_race': 1}]
unpriv_groups = [{'driver_race': 0}]
dataset = pd.concat([X_test, y_test], axis=1)
dataset = StandardDataset(dataset, label_name='is_arrested', favorable_classes=[0], protected_attribute_names=['driver_race'], privileged_classes=[[1]])
classified_dataset = dataset.copy()
classified_dataset.labels = y_pred

# metrics
class_metric = ClassificationMetric(dataset, classified_dataset, unprivileged_groups=unpriv_groups, privileged_groups=priv_groups)
print(  
    ' FOR:', class_metric.false_omission_rate(privileged=1),'\n',
    'FORD:', class_metric.false_omission_rate_difference(),'\n',
    'Accuracy score:', advd.score(X_test, y_test)
    )

 FOR: 0.5946197740770933 
 FORD: -0.2680891618321954 
 Accuracy score: 0.5938570516170237


# Logistic regression with postprocessing (no artificial bias)

## Without unbiasing

In [43]:
import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
from sklearn.model_selection import  train_test_split
from sklearn.linear_model import LogisticRegression
from aif360.metrics import ClassificationMetric
from aif360.datasets import StandardDataset, dataset
from copy import copy
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
_night = ['22:', '23:', '00:', '01:', '02:', '03:', '04:', '05:', '06:', '07:']

# data preprocessing
data = pd.read_csv('data/police_project.csv')
data = data.drop(['stop_date','county_name', 'driver_age_raw', 'violation_raw', 'stop_outcome', 'drugs_related_stop', 'search_type'], axis=1)
data = data.dropna()
data['stop_time'] = data['stop_time'].apply(lambda x: 0 if any(n in x for n in _night) else 1)
data['driver_gender'] = data['driver_gender'].astype('category').cat.codes
data['driver_race'] = data['driver_race'].astype('category').cat.codes
data['driver_race'] = data['driver_race'].apply(lambda x: 0 if x==3 else 1)
data['violation'] = data['violation'].astype('category').cat.codes
data['search_conducted'] = data['search_conducted'].astype('category').cat.codes
data['is_arrested'] = data['is_arrested'].astype('category').cat.codes # 0 = False, 1 = True
data['stop_duration'] = data['stop_duration'].astype('category').cat.codes

# datasets
X = data[data.columns.difference(['is_arrested'])]
y = data['is_arrested']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = LogisticRegression(max_iter=1000, ).fit(X_train, y_train)
y_pred = clf.predict(X_test)

# ai360
priv_groups = [{'driver_race': 1}]
unpriv_groups = [{'driver_race': 0}]
dataset = pd.concat([X_test, y_test], axis=1)
dataset = StandardDataset(dataset, label_name='is_arrested', favorable_classes=[0], protected_attribute_names=['driver_race'], privileged_classes=[[1]])
classified_dataset = dataset.copy()
classified_dataset.labels = y_pred

# metrics
class_metric = ClassificationMetric(dataset, classified_dataset, unprivileged_groups=unpriv_groups, privileged_groups=priv_groups)
print(  
    ' FOR:', class_metric.false_omission_rate(privileged=1),'\n',
    'FORD:', class_metric.false_omission_rate_difference(),'\n',
    'Accuracy score:', clf.score(X_test, y_test)
    )

 FOR: 0.5253164556962026 
 FORD: -0.5253164556962026 
 Accuracy score: 0.9660918539162747


invalid value encountered in double_scalars
invalid value encountered in double_scalars


## With unbiasing

In [46]:
from aif360.sklearn.postprocessing import CalibratedEqualizedOdds

coln = ['driver_race'] # stupid workaround for make it working
X_test.index.names = coln
ceo = CalibratedEqualizedOdds(prot_attr='driver_race').fit(X_test, pd.Series(y_pred))

ValueError: Some of the attributes provided are not present in the dataset. Expected a subset of:
[]
Got:
['driver_race']

In [ ]:
# datasets
X = data[data.columns.difference(['is_arrested'])]
y = data['is_arrested']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = LogisticRegression(class_weight=cweights, max_iter=1000).fit(X_train, y_train)
y_pred = clf.predict(X_test)

# ai360
priv_groups = [{'driver_race': 1}]
unpriv_groups = [{'driver_race': 0}]
dataset = pd.concat([X_test, y_test], axis=1)
dataset = StandardDataset(dataset, label_name='is_arrested', favorable_classes=[0], protected_attribute_names=['driver_race'], privileged_classes=[[1]])
classified_dataset = dataset.copy()
classified_dataset.labels = y_pred

# metrics
class_metric = ClassificationMetric(dataset, classified_dataset, unprivileged_groups=unpriv_groups, privileged_groups=priv_groups)
print(  
    ' FOR:', class_metric.false_omission_rate(privileged=1),'\n',
    'FORD:', class_metric.false_omission_rate_difference(),'\n',
    'Accuracy score:', clf.score(X_test, y_test)
    )